In [ ]:
# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from pathlib import Path
import gc

# Check GPU availability
print("TensorFlow version:", tf.__version__)
print("TFDS version:", tfds.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Set memory growth for GPU (prevents OOM errors)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

# Optimize CPU threads to prevent overload
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)
print("✓ CPU threading optimized")

In [ ]:
# ============================================================================
# CELL 2: CONFIGURATION AND HYPERPARAMETERS (OPTIMIZED)
# ============================================================================

# Paths for saving models and results
SAVE_DIR = "/content/drive/MyDrive/cnn/transfer_function"

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

# Image and training parameters - OPTIMIZED FOR CPU
IMAGE_SIZE = 224
BATCH_SIZE = 16  # Reduced from 32
NUM_WORKERS = 2  # Limited parallel workers
PREFETCH_BUFFER = 2  # Reduced from AUTOTUNE

# Training hyperparameters - ADJUSTED
EPOCHS_PHASE1 = 12  # Reduced slightly
EPOCHS_PHASE2 = 15  # Reduced slightly
LEARNING_RATE_P1 = 1e-3
LEARNING_RATE_P2 = 1e-5

# Data loading optimization
CACHE_ENABLED = False  # Disable cache for large dataset to save RAM
SHUFFLE_BUFFER = 1000  # Reduced shuffle buffer

print("\n" + "="*60)
print("OPTIMIZED CONFIGURATION FOR 55K DATASET")
print("="*60)
print(f"Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Batch Size: {BATCH_SIZE} (optimized)")
print(f"Num Workers: {NUM_WORKERS} (limited)")
print(f"Prefetch Buffer: {PREFETCH_BUFFER} (controlled)")
print(f"Shuffle Buffer: {SHUFFLE_BUFFER}")
print(f"Cache Enabled: {CACHE_ENABLED}")
print(f"Phase 1 Epochs: {EPOCHS_PHASE1}")
print(f"Phase 2 Epochs: {EPOCHS_PHASE2}")

In [ ]:
# ============================================================================
# CELL 3: LOAD PLANTVILLAGE DATASET FROM TFDS
# ============================================================================

print("\n" + "="*60)
print("LOADING PLANTVILLAGE DATASET FROM TENSORFLOW DATASETS")
print("="*60)

# Load dataset with optimized settings
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'plant_village',
    split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    download=True,
    try_gcs=True,  # Try Google Cloud Storage first for faster download
)

print("\n✓ Dataset loaded successfully!")
print(f"\n{ds_info}")

# Get dataset information
NUM_CLASSES = ds_info.features['label'].num_classes
CLASS_NAMES = ds_info.features['label'].names
TOTAL_IMAGES = ds_info.splits['train'].num_examples
TRAIN_SIZE = int(TOTAL_IMAGES * 0.7)
VAL_SIZE = int(TOTAL_IMAGES * 0.15)
TEST_SIZE = TOTAL_IMAGES - TRAIN_SIZE - VAL_SIZE

print("\n" + "="*60)
print("DATASET INFORMATION")
print("="*60)
print(f"Total Images: {TOTAL_IMAGES}")
print(f"Training Images: {TRAIN_SIZE} (70%)")
print(f"Validation Images: {VAL_SIZE} (15%)")
print(f"Test Images: {TEST_SIZE} (15%)")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"Steps per epoch: {TRAIN_SIZE // BATCH_SIZE}")

In [ ]:
# ============================================================================
# CELL 4: DATA PREPROCESSING AND AUGMENTATION (OPTIMIZED)
# ============================================================================

print("\n" + "="*60)
print("SETTING UP OPTIMIZED DATA PREPROCESSING")
print("="*60)

# Simplified data augmentation to reduce CPU load
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),  # Reduced from 0.2
    layers.RandomZoom(0.15),  # Reduced from 0.2
], name='data_augmentation')

# Preprocessing function
@tf.function  # Compile to graph for faster execution
def preprocess_image(image, label):
    """Resize and normalize images - optimized"""
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1.0
    return image, label

@tf.function  # Compile to graph
def augment_and_preprocess(image, label):
    """Apply augmentation and preprocessing for training - optimized"""
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = data_augmentation(image, training=True)
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1.0
    return image, label

# Apply preprocessing with controlled parallelism
print("\n✓ Applying optimized preprocessing pipeline...")

# Training dataset - most optimized
ds_train_processed = (
    ds_train
    .shuffle(SHUFFLE_BUFFER, reshuffle_each_iteration=True)
    .map(augment_and_preprocess, num_parallel_calls=NUM_WORKERS)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(PREFETCH_BUFFER)
)

# Validation dataset - lighter processing
ds_val_processed = (
    ds_val
    .map(preprocess_image, num_parallel_calls=NUM_WORKERS)
    .batch(BATCH_SIZE)
    .prefetch(PREFETCH_BUFFER)
)

# Test dataset - no augmentation
ds_test_processed = (
    ds_test
    .map(preprocess_image, num_parallel_calls=NUM_WORKERS)
    .batch(BATCH_SIZE)
    .prefetch(PREFETCH_BUFFER)
)

print(f"✓ Training batches: {TRAIN_SIZE // BATCH_SIZE}")
print(f"✓ Validation batches: {VAL_SIZE // BATCH_SIZE}")
print(f"✓ Test batches: {TEST_SIZE // BATCH_SIZE}")
print("✓ Pipeline optimized for CPU efficiency")




In [ ]:
# ============================================================================
# CELL 5: HELPER FUNCTIONS
# ============================================================================

def create_callbacks(filepath, patience_reduce=3, patience_early=8, min_lr=1e-7):
    """Create training callbacks - adjusted patience"""

    checkpoint = ModelCheckpoint(
        filepath=os.path.join(SAVE_DIR, filepath),
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=patience_reduce,
        min_lr=min_lr,
        verbose=1,
        mode='min'
    )

    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=patience_early,  # Reduced from 10
        restore_best_weights=True,
        verbose=1,
        mode='max'
    )

    return [checkpoint, reduce_lr, early_stopping]


def plot_training_history(history, model_name, phase=""):
    """Plot training and validation metrics"""

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Accuracy plot
    axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0].set_title(f'{model_name} {phase} - Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Loss plot
    axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[1].set_title(f'{model_name} {phase} - Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f'{model_name}_{phase}_history.png'), dpi=150, bbox_inches='tight')
    plt.show()
    plt.close()  # Free memory


def evaluate_model(model, test_dataset, model_name):
    """Evaluate model on test set"""
    print(f"\n{'='*60}")
    print(f"EVALUATING {model_name}")
    print(f"{'='*60}")

    results = model.evaluate(test_dataset, verbose=1)

    print(f"\n✓ Test Loss: {results[0]:.4f}")
    print(f"✓ Test Accuracy: {results[1]:.4f}")

    return results


def compile_model(model, learning_rate=1e-4):
    """Compile model with standard configuration"""
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


def clear_memory():
    """Clear memory between models"""
    keras.backend.clear_session()
    gc.collect()
    print("✓ Memory cleared")

In [ ]:
# ============================================================================
# CELL 6: MODEL 1 - ResNet50 (2-Phase Training)
# ============================================================================

print("\n" + "="*60)
print("MODEL 1: ResNet50 WITH 2-PHASE TRAINING")
print("="*60)

clear_memory()

# Load ResNet50 base model
base_model_resnet50 = tf.keras.applications.ResNet50(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model_resnet50.trainable = False

print(f"\n✓ ResNet50 loaded (frozen)")
print(f"✓ Total layers in base model: {len(base_model_resnet50.layers)}")

# Build model with dropout adjusted
input_layer = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='input')
x = base_model_resnet50(input_layer, training=False)
x = layers.GlobalAveragePooling2D(name='gap')(x)
x = layers.Dense(512, activation='relu', name='dense_512')(x)
x = layers.BatchNormalization(name='bn_1')(x)
x = layers.Dropout(0.4, name='dropout_1')(x)  # Increased dropout
x = layers.Dense(256, activation='relu', name='dense_256')(x)
x = layers.BatchNormalization(name='bn_2')(x)
x = layers.Dropout(0.4, name='dropout_2')(x)
x = layers.Dense(128, activation='relu', name='dense_128')(x)
x = layers.Dropout(0.3, name='dropout_3')(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model_resnet50 = Model(inputs=input_layer, outputs=outputs, name='ResNet50_Classifier')

# Compile model
model_resnet50 = compile_model(model_resnet50, learning_rate=LEARNING_RATE_P1)

print("\n✓ Model built and compiled")

In [ ]:
# ============================================================================
# CELL 7: Train ResNet50 - Phase 1 (Frozen Base)
# ============================================================================

print("\n" + "="*60)
print("TRAINING ResNet50 - PHASE 1 (FROZEN BASE MODEL)")
print("="*60)

callbacks_resnet50_p1 = create_callbacks('best_resnet50_phase1.h5', patience_reduce=3, patience_early=7, min_lr=1e-7)

start_time_resnet50 = time.time()

history_resnet50_phase1 = model_resnet50.fit(
    ds_train_processed,
    epochs=EPOCHS_PHASE1,
    validation_data=ds_val_processed,
    callbacks=callbacks_resnet50_p1,
    verbose=1,
    workers=1,  # Single worker to reduce CPU load
    use_multiprocessing=False  # Disable multiprocessing
)

phase1_time_resnet50 = time.time() - start_time_resnet50
print(f"\n✓ Phase 1 completed in {phase1_time_resnet50/60:.2f} minutes")

# Plot Phase 1 history
plot_training_history(history_resnet50_phase1, 'ResNet50', 'Phase1')



In [ ]:

# ============================================================================
# CELL 8: Train ResNet50 - Phase 2 (Fine-tuning)
# ============================================================================

print("\n" + "="*60)
print("TRAINING ResNet50 - PHASE 2 (FINE-TUNING)")
print("="*60)

# Unfreeze the base model
base_model_resnet50.trainable = True

# Freeze all layers except the last 15 (reduced from 20)
for layer in base_model_resnet50.layers[:-15]:
    layer.trainable = False

print(f"✓ Unfrozen last 15 layers of ResNet50")
print(f"✓ Trainable layers: {sum([1 for layer in model_resnet50.layers if layer.trainable])}")

# Recompile with lower learning rate
model_resnet50 = compile_model(model_resnet50, learning_rate=LEARNING_RATE_P2)

callbacks_resnet50_p2 = create_callbacks('best_resnet50_phase2.h5', patience_reduce=2, patience_early=6, min_lr=1e-7)

history_resnet50_phase2 = model_resnet50.fit(
    ds_train_processed,
    epochs=EPOCHS_PHASE2,
    validation_data=ds_val_processed,
    callbacks=callbacks_resnet50_p2,
    verbose=1,
    workers=1,
    use_multiprocessing=False
)

phase2_time_resnet50 = time.time() - start_time_resnet50 - phase1_time_resnet50
total_training_time_resnet50 = time.time() - start_time_resnet50

print(f"\n✓ Phase 2 completed in {phase2_time_resnet50/60:.2f} minutes")
print(f"✓ Total training time: {total_training_time_resnet50/60:.2f} minutes")

# Plot Phase 2 history
plot_training_history(history_resnet50_phase2, 'ResNet50', 'Phase2')

# Evaluate on test set
resnet50_results = evaluate_model(model_resnet50, ds_test_processed, 'ResNet50')

# Save model
model_resnet50.save(os.path.join(SAVE_DIR, 'PlantVillage_ResNet50_final.h5'))
print("\n✓ ResNet50 model saved!")

In [ ]:
# ============================================================================
# CELL 9: MODEL 2 - ResNet101 (Full Fine-tuning)
# ============================================================================

print("\n" + "="*60)
print("MODEL 2: ResNet101 FULL FINE-TUNING")
print("="*60)

clear_memory()

# Load ResNet101 base model
base_model_resnet101 = tf.keras.applications.ResNet101(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Make all layers trainable
base_model_resnet101.trainable = True

print(f"\n✓ ResNet101 loaded (all layers trainable)")
print(f"✓ Total layers: {len(base_model_resnet101.layers)}")

# Build model - simplified
input_layer = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model_resnet101(input_layer, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model_resnet101 = Model(inputs=input_layer, outputs=outputs, name='ResNet101_FineTuning')

# Compile with very low learning rate
model_resnet101 = compile_model(model_resnet101, learning_rate=1e-5)

print("\n✓ Model built and compiled")


In [ ]:
# ============================================================================
# CELL 10: Train ResNet101
# ============================================================================

print("\n" + "="*60)
print("TRAINING ResNet101 - FULL FINE-TUNING")
print("="*60)

callbacks_resnet101 = create_callbacks('best_resnet101.h5', patience_reduce=3, patience_early=7, min_lr=1e-7)

start_time_resnet101 = time.time()

history_resnet101 = model_resnet101.fit(
    ds_train_processed,
    epochs=15,  # Reduced from 20
    validation_data=ds_val_processed,
    callbacks=callbacks_resnet101,
    verbose=1,
    workers=1,
    use_multiprocessing=False
)

training_time_resnet101 = time.time() - start_time_resnet101
print(f"\n✓ Training completed in {training_time_resnet101/60:.2f} minutes")

# Plot history
plot_training_history(history_resnet101, 'ResNet101', 'FullFineTuning')

# Evaluate on test set
resnet101_results = evaluate_model(model_resnet101, ds_test_processed, 'ResNet101')

# Save final model
model_resnet101.save(os.path.join(SAVE_DIR, 'PlantVillage_Resnet101_FineTuning.h5')) # keras yapmamızı istedi
print("\n✓ ResNet101 model saved!")

In [ ]:
# ============================================================================
# CELL 11: MODEL 3 - EfficientNetB0 (Lightweight Alternative)
# ============================================================================

print("\n" + "="*60)
print("MODEL 3: EfficientNetB0 (LIGHTWEIGHT)")
print("="*60)

clear_memory()

# Load EfficientNetB0
base_model_effnet = tf.keras.applications.EfficientNetB0(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model_effnet.trainable = False

print(f"\n✓ EfficientNetB0 loaded (frozen)")
print(f"✓ Total layers: {len(base_model_effnet.layers)}")

# Build model
input_layer = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model_effnet(input_layer, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model_effnet = Model(inputs=input_layer, outputs=outputs, name='EfficientNetB0')

# Compile
model_effnet = compile_model(model_effnet, learning_rate=LEARNING_RATE_P1)

print("\n✓ Model built and compiled")

# Train Phase 1
print("\n" + "="*60)
print("TRAINING EfficientNetB0 - PHASE 1")
print("="*60)

callbacks_effnet_p1 = create_callbacks('best_effnet_phase1.h5', patience_reduce=3, patience_early=7, min_lr=1e-7)

start_time_effnet = time.time()

history_effnet_phase1 = model_effnet.fit(
    ds_train_processed,
    epochs=EPOCHS_PHASE1,
    validation_data=ds_val_processed,
    callbacks=callbacks_effnet_p1,
    verbose=1,
    workers=1,
    use_multiprocessing=False
)

phase1_time_effnet = time.time() - start_time_effnet
print(f"\n✓ Phase 1 completed in {phase1_time_effnet/60:.2f} minutes")

# Fine-tuning Phase 2
print("\n" + "="*60)
print("TRAINING EfficientNetB0 - PHASE 2")
print("="*60)

base_model_effnet.trainable = True
for layer in base_model_effnet.layers[:-25]:  # Reduced from 30
    layer.trainable = False

model_effnet = compile_model(model_effnet, learning_rate=LEARNING_RATE_P2)

callbacks_effnet_p2 = create_callbacks('best_effnet_phase2.h5', patience_reduce=2, patience_early=6, min_lr=1e-7)

history_effnet_phase2 = model_effnet.fit(
    ds_train_processed,
    epochs=EPOCHS_PHASE2,
    validation_data=ds_val_processed,
    callbacks=callbacks_effnet_p2,
    verbose=1,
    workers=1,
    use_multiprocessing=False
)

total_time_effnet = time.time() - start_time_effnet
print(f"\n✓ Total training time: {total_time_effnet/60:.2f} minutes")

# Plot and evaluate
plot_training_history(history_effnet_phase1, 'EfficientNetB0', 'Phase1')
plot_training_history(history_effnet_phase2, 'EfficientNetB0', 'Phase2')
effnet_results = evaluate_model(model_effnet, ds_test_processed, 'EfficientNetB0')

# Save model
model_effnet.save(os.path.join(SAVE_DIR, 'PlantVillage_EfficientNetB0_final.h5'))
print("\n✓ EfficientNetB0 model saved!")

In [ ]:
# ============================================================================
# CELL 12: FINAL COMPARISON AND RESULTS
# ============================================================================

print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)

# Compare all models
models_comparison = {
    'ResNet50': model_resnet50,
    'ResNet101': model_resnet101,
    'EfficientNetB0': model_effnet
}

results_summary = {}

for name, model in models_comparison.items():
    print(f"\nEvaluating {name}...")
    results = model.evaluate(ds_test_processed, verbose=0)
    results_summary[name] = {
        'Test Loss': results[0],
        'Test Accuracy': results[1]
    }

# Print summary table
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)
print(f"{'Model':<20} {'Test Loss':<15} {'Test Accuracy':<15}")
print("-" * 60)
for name, metrics in results_summary.items():
    print(f"{name:<20} {metrics['Test Loss']:<15.4f} {metrics['Test Accuracy']:<15.4f}")

# Find best model
best_model_name = max(results_summary, key=lambda x: results_summary[x]['Test Accuracy'])
print(f"\n✓ Best Model: {best_model_name}")
print(f"✓ Best Accuracy: {results_summary[best_model_name]['Test Accuracy']:.4f}")

print("\n" + "="*60)
print("TRAINING COMPLETED!")
print("="*60)

# Final cleanup
clear_memory()